In [1]:
import numpy as np 
import pandas as pd 
import csv
import re

In [8]:
#Notebook crashes if all rows are allowed to display 
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

Hyposisis that the following data can be incorrect due to the registration form shortfalls.

- Companies does not check if Company name is a trade mark. Not sure if they allow only one company per name. We may companies with exsact names?
- Postcodes in the new form are now valideted but not full dresses if address is provided manually. Do we have companies with valid postcodes but not addresses that were regestered recently?
- Countries are now restricted to UK only. No foreign companies registered recently? 

https://www.gov.uk/government/publications/incorporation-and-names/annex-a-sensitive-words-and-expressions-or-words-that-could-imply-a-connection-with-government

In [3]:
df = pd.read_csv('BasicCompanyDataAsOneFile-2020-07-01.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (41,42,43,44,45,46,47,48,49,50,51,52) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head(100)

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,...,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate
0,! LTD,08209948,NaN,NaN,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,NaN,LS10 2RU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23/10/2020,11/09/2019
1,!? LTD,11399177,NaN,NaN,THE STUDIO HATHERLOW HOUSE,HATHERLOW,ROMILEY,NaN,UNITED KINGDOM,SK6 3DY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19/06/2021,05/06/2020
2,!BIG IMPACT GRAPHICS LIMITED,11743365,NaN,NaN,372 OLD STREET,335 ROSDEN HOUSE,LONDON,NaN,UNITED KINGDOM,EC1V 9LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/02/2021,27/12/2019
3,!L PRODUCTIONS LIMITED,12402527,NaN,NaN,95 MARMION AVENUE,NaN,CHINGFORD,NaN,ENGLAND,E4 8EJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24/02/2021,NaN
4,!NKED LTD,12234705,NaN,NaN,29 CORRY DRIVE,NaN,LONDON,NaN,UNITED KINGDOM,SW9 8QS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/11/2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"""JOHNNIE"" JOHNSON HOUSING TRUST LIMITED",IP19198R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15/01/2017,NaN
96,"""K"" LAUNDRY LIMITED",00226797,NaN,NaN,"K"" LAUNDRY",NORTHWOOD,RAMSGATE,KENT,NaN,CT12 6RR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24/02/2021,13/01/2020
97,"""K"" LINE (EUROPE) LIMITED",05639474,NaN,NaN,200 ALDERSGATE STREET,NaN,LONDON,NaN,NaN,EC1A 4HD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/01/2021,29/11/2019
98,"""K"" LINE BULK SHIPPING (UK) LIMITED",04830352,NaN,NaN,200 ALDERSGATE STREET,NaN,LONDON,NaN,NaN,EC1A 4HD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/08/2020,27/06/2019


Lets review entries in column RegAddress.Country and make sure that they make sense and that we are not useing diffrent names for the same country. 

In [14]:
df['RegAddress.Country'].unique().tolist()

[nan,
 'UNITED KINGDOM',
 'ENGLAND',
 'SCOTLAND',
 'GUERNSEY',
 'MONGOLIA',
 'GERMANY',
 'POLAND',
 'WALES',
 'ISRAEL',
 'EGYPT',
 'AUSTRIA',
 'KAZAKHSTAN',
 'ROMANIA',
 'BRAZIL',
 'CROATIA',
 'UNITED STATES',
 'CANADA',
 'NORTHERN IRELAND',
 'JERSEY',
 'MALAYSIA',
 'VIRGIN ISLANDS',
 'CAYMAN ISLANDS',
 'CHANNEL ISLANDS',
 'NETHERLANDS',
 'AUSTRALIA',
 'SWITZERLAND',
 'INDIA',
 'FRANCE',
 'IRELAND',
 'SRI LANKA',
 'GHANA',
 'ITALY',
 'GREECE',
 'NIGERIA',
 'UNITED ARAB EMIRATES',
 'DENMARK',
 'BANGLADESH',
 'VIRGIN ISLANDS, BRITISH',
 'ISLE OF MAN',
 'JAPAN',
 'GIBRALTAR',
 'SINGAPORE',
 'PAKISTAN',
 'BAHAMAS',
 'HONG KONG',
 'BERMUDA',
 'RUSSIA',
 'SPAIN',
 'BELIZE',
 'SOUTH KOREA',
 'BELGIUM',
 'URUGUAY',
 'CHINA',
 'NORWAY',
 'SWEDEN',
 'ARGENTINA',
 'MEXICO',
 'LIECHTENSTEIN',
 'SEYCHELLES',
 'IRAN',
 'MOZAMBIQUE',
 'SOUTH AFRICA',
 'TANZANIA',
 'KENYA',
 'MALAWI',
 'RWANDA',
 'UGANDA',
 'ZAMBIA',
 'CZECHOSLOVAKIA',
 'IVORY COAST',
 'ALGERIA',
 'JAMAICA',
 'MALTA',
 'MAURITIUS',
 '

In [20]:
df.loc[df['RegAddress.Country'] == 'ENGLAND', ['RegAddress.Country']] = 'UNITED KINGDOM'
df.loc[df['RegAddress.Country'] == 'SCOTLAND', ['RegAddress.Country']] = 'UNITED KINGDOM'
df.loc[df['RegAddress.Country'] == 'WALES', ['RegAddress.Country']] = 'UNITED KINGDOM'
df.loc[df['RegAddress.Country'] == 'LIVERPOOL', ['RegAddress.Country']] = 'UNITED KINGDOM'
df.loc[df['RegAddress.Country'] == 'PO33 2TG', ['RegAddress.Country']] = 'UNITED KINGDOM'
df.loc[df['RegAddress.Country'] == 'WEST YORKSHIRE', ['RegAddress.Country']] = 'UNITED KINGDOM'
df.loc[df['RegAddress.Country'] == 'ENGLAND & WALES', ['RegAddress.Country']] = 'UNITED KINGDOM'
df.loc[df['RegAddress.Country'] == 'SCOTLAND  UK', ['RegAddress.Country']] = 'UNITED KINGDOM'
df.loc[df['RegAddress.Country'] == 'SW7', ['RegAddress.Country']] = 'UNITED KINGDOM'

In [23]:
# something odd is happening since we have countries that are based in the USSR
# USSR does not exsist for 20 years
# How come we have companies registered there?
df.loc[df['RegAddress.Country'] == 'USSR']

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,CompanyCategory,CompanyStatus,CountryOfOrigin,DissolutionDate,IncorporationDate,Accounts.AccountRefDay,Accounts.AccountRefMonth,Accounts.NextDueDate,Accounts.LastMadeUpDate,Accounts.AccountCategory,Returns.NextDueDate,Returns.LastMadeUpDate,Mortgages.NumMortCharges,Mortgages.NumMortOutstanding,Mortgages.NumMortPartSatisfied,Mortgages.NumMortSatisfied,SICCode.SicText_1,SICCode.SicText_2,SICCode.SicText_3,SICCode.SicText_4,LimitedPartnerships.NumGenPartners,LimitedPartnerships.NumLimPartners,URI,PreviousName_1.CONDATE,PreviousName_1.CompanyName,PreviousName_2.CONDATE,PreviousName_2.CompanyName,PreviousName_3.CONDATE,PreviousName_3.CompanyName,PreviousName_4.CONDATE,PreviousName_4.CompanyName,PreviousName_5.CONDATE,PreviousName_5.CompanyName,PreviousName_6.CONDATE,PreviousName_6.CompanyName,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate,Reg_Country
969647,"COMMERCIAL BANK ""MOST-BANK",FC018405,NaN,NaN,DR MICHAEL V. BORSHCHEVSKY,32 RINGMORE RISE,LONDON,NaN,USSR,SE23 3DE,Other company type,Active,USSR,NaN,29/12/1994,31.0,12.0,NaN,31/12/1998,FULL,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/FC018405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/01/2017,NaN,USSR
969650,"COMMERCIAL BANK OF PROMOTION OF PROGRESSIVE TECHNOLOGIES AND INVESTMENTS IN THE FIELD OF POWER INDUSTRY, BIOTECHNOLOGY AND MEDICINE (""ELBIM-BANK"")",FC018634,NaN,NaN,MR SERGEI MOROZOV (+ 1 OTHER),104 SALISBURY HOUSE,LONDON WALL,LONDON,USSR,EC2M 5QQ,Other company type,Active,USSR,NaN,01/05/1995,31.0,12.0,NaN,31/12/1998,FULL,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/FC018634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15/05/2017,NaN,USSR
969652,COMMERCIAL BANK STROYINVEST,FC017697,NaN,NaN,L.P. BLAKELEY,COMMERCIAL BANK STROYINVEST,"SUITE 252, 2 LANSDOWNE ROW",BERKELEY SQUARE LONDON8HL,USSR,NaN,Other company type,Active,USSR,NaN,13/12/1993,31.0,12.0,NaN,01/01/1999,FULL,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/FC017697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27/12/2016,NaN,USSR
983371,CONCERN MILAN,FC019167,NaN,NaN,45 LENINSKY PROSPECT,MOSCOW,RUSSIA,NaN,USSR,NaN,Other company type,Active,USSR,NaN,12/04/1996,30.0,4.0,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/FC019167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21/05/2017,NaN,USSR
1400849,ENERGY MACHINE BUILDING BANK (ENERGOMASHBANK) (PUBLIC JOINT STOCK COMPANY),FC020646,NaN,NaN,MR ALEXEI FOMINE,276 HIGH STREET,GUILDFORD,SURREY,USSR,GU1 3JL,Other company type,Active,USSR,NaN,26/09/1997,31.0,12.0,NaN,31/12/2001,FULL,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/FC020646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/10/2016,NaN,USSR
1509811,"FEDERAL STATE UNITARY ENTERPRISE ""STATE TRANSPORT COMPANY ""RUSSIA"" T/A - ""ROSSIYA"" - RUSSIAN AIRLINES",FC025213,NaN,NaN,ALEXANDRE SHUKLIN,THE MEWS,ST NICHOLAS LANE,LEWES LEWES EAST SUSSEX,USSR,BN7 2JZ,Other company type,Active,USSR,NaN,21/04/2004,30.0,4.0,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/FC025213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05/05/2017,NaN,USSR
2086622,"INTER-REGIONAL PUBLIC ORGANIZATION ""THE CENTER FOR INTERNATIONAL ECONOMIC SCIENTIFIC AND CULTURAL COOPERATION""",F

In [5]:
# manual review of the most common and least common postcodes
# no problematic postcodes are being selected this way
#df['RegAddress.PostCode'].value_counts().to_frame().tail(1000)
df['RegAddress.PostCode'].value_counts().to_frame().head(1000)
#df['RegAddress.PostCode'].value_counts().to_frame()

,RegAddress.PostCode
N1 7GU,42607
WC2H 9JQ,36826
EC1V 2NX,28174
WA1 1RG,13963
WC1N 3AX,13121
...,...
LE12 7NN,329
SK9 4LY,329
M17 8PL,328
NN1 5PT,328


In [6]:
# adding column that contains first part of postcode split by space
new = df['RegAddress.PostCode'].str.split(" ", n = 1, expand = True) 
df["PostCode1"]= new[0] 
#df_postcode["PostCode2"]= new[1] 
df

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,...,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate,PostCode1
0,! LTD,08209948,NaN,NaN,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,NaN,LS10 2RU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23/10/2020,11/09/2019,LS10
1,!? LTD,11399177,NaN,NaN,THE STUDIO HATHERLOW HOUSE,HATHERLOW,ROMILEY,NaN,UNITED KINGDOM,SK6 3DY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19/06/2021,05/06/2020,SK6
2,!BIG IMPACT GRAPHICS LIMITED,11743365,NaN,NaN,372 OLD STREET,335 ROSDEN HOUSE,LONDON,NaN,UNITED KINGDOM,EC1V 9LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/02/2021,27/12/2019,EC1V
3,!L PRODUCTIONS LIMITED,12402527,NaN,NaN,95 MARMION AVENUE,NaN,CHINGFORD,NaN,ENGLAND,E4 8EJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24/02/2021,NaN,E4
4,!NKED LTD,12234705,NaN,NaN,29 CORRY DRIVE,NaN,LONDON,NaN,UNITED KINGDOM,SW9 8QS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/11/2020,NaN,SW9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4671961,“THE GREENHOUSE” COMPANY LTD,11457383,NaN,NaN,IRONMASTER HOUSE,WYLE COP,SHREWSBURY,SHROPSHIRE,ENGLAND,SY1 1XF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/07/2021,25/06/2020,SY1
4671962,“THE SKULL” SAUCE COMPANY LTD,SC660098,NaN,NaN,40 CLAYKNOWES PLACE,NaN,MUSSELBURGH,NaN,SCOTLAND,EH21 6UQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/05/2021,NaN,EH21
4671963,€URI UK LIMITED,11382243,NaN,NaN,1 MANN ISLAND,NaN,LIVERPOOL,MERSEYSIDE,NaN,L3 1BP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/06/2019,NaN,L3
4671964,€URO IMPORTS LIMITED,08182582,NaN,NaN,414-416 BLACKPOOL ROAD,ASHTON-ON-RIBBLE,PRESTON,NaN,NaN,PR2 2DX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/09/2020,14/08/2019,PR2


In [9]:
# manually checking that split worked correctly
df[['RegAddress.PostCode', 'PostCode1']].head(2000)

,RegAddress.PostCode,PostCode1
0,LS10 2RU,LS10
1,SK6 3DY,SK6
2,EC1V 9LT,EC1V
3,E4 8EJ,E4
4,SW9 8QS,SW9
5,TN8 5NF,TN8
6,AB11 7SY,AB11
7,AB11 7SY,AB11
8,W12 8DS,W12
9,GY1 1EW,GY1


In [15]:
# Checking where outercodes/values in column PostCode1 occure i the data set less then 20 times
# We are not expecting incorrect outercodes to appear frequently 
vc = df['PostCode1'].value_counts().head(3000)
vc[vc<20]

IM8         19
L29         19
PA22        19
EH38        19
PH50        19
C07         19
PA33        19
LL51        19
LL43        19
EC2         19
SO5         18
KA14        18
PH31        18
IV44        18
BS19        18
BT64        17
C09         17
RG13        17
W15         17
LL37        17
BS17        17
PH5         17
PA65        17
IM99        16
PH35        16
CF7         16
EC3         16
YO4         16
PH39        16
PA60        16
IV52        16
PE18        16
SO9         16
IV42        16
SSO         15
IV43        15
PA26        15
PA44        15
RG3         15
EC4         15
EC1P        15
PA72        15
PA48        14
NP5         14
PA61        14
NP3         14
HAO         14
CA18        14
B40         14
NG90        14
LD4         14
BS12        14
PA76        14
IV45        13
JE3         13
LL27        13
S30         13
IV41        13
C04         13
GY4         13
FK20        12
KY1-1104    12
PH40        12
LL38        12
AB2         12
DE2         12
S31       

In [24]:
# Another way to review incorrect outercodes is to check rows where column PostCode1 has values that are longer 
# than 4 charecters.
# UNITED KINGDOM Look at the ones that are in UK and the ones that are not.
# Check if more country entries can be converted to UNITED KINGDOM
df[['PostCode1', 'RegAddress.Country', 'RegAddress.PostCode']][(df['PostCode1'].str.len()>4)]

,PostCode1,RegAddress.Country,RegAddress.PostCode
962,XXXXXXXX,NaN,XXXXXXXX
1021,XXXXXXXXX,NaN,XXXXXXXXX
3199,SW1X9SH,UNITED KINGDOM,SW1X9SH
9562,3453JG,NaN,3453JG
15209,TRH10,NaN,TRH10 1NH
18475,RH162NL,UNITED KINGDOM,RH162NL
18918,BH170GB,UNITED KINGDOM,BH170GB
31366,BL13HT,UNITED KINGDOM,BL13HT
34253,OX41XX,UNITED KINGDOM,OX41XX
82446,WA51DA,UNITED KINGDOM,WA51DA


In [25]:
# closer look at the incorrect outercodes where country is UK
df[['PostCode1', 'RegAddress.Country', 'RegAddress.PostCode']][(df['PostCode1'].str.len()>4) & (df['RegAddress.Country'] == 'UNITED KINGDOM')]

,PostCode1,RegAddress.Country,RegAddress.PostCode
3199,SW1X9SH,UNITED KINGDOM,SW1X9SH
18475,RH162NL,UNITED KINGDOM,RH162NL
18918,BH170GB,UNITED KINGDOM,BH170GB
31366,BL13HT,UNITED KINGDOM,BL13HT
34253,OX41XX,UNITED KINGDOM,OX41XX
82446,WA51DA,UNITED KINGDOM,WA51DA
123676,EC2M3TY,UNITED KINGDOM,EC2M3TY
124756,NE258EP,UNITED KINGDOM,NE258EP
139522,GL69HH,UNITED KINGDOM,GL69HH
148515,PL158TL,UNITED KINGDOM,PL158TL


In [31]:
df['Postcode_FX'] = df['RegAddress.PostCode'].replace(" ","").str.upper()

df['Postcode_Fixed'] = df['Postcode_FX'].str.findall(r'^((([A-Z][A-Z]{0,1})([0-9][A-Z0-9]{0,2})) {0,}(([0-9])([A-Z]{2})))')

#df['Outercode'] = df['Postcode_FX'][1]

In [40]:
def label_race (row):
   try:
       if(type(row['Postcode_Fixed'])==list):
        return row['Postcode_Fixed'][1]
       return ''
   except IndexError:
       pass

In [ ]:
df.apply (lambda row: label_race(row), axis=1)

In [26]:
#df['RegAddress.PostCode'].unique().tolist()

In [32]:
df[['RegAddress.PostCode','Postcode_FX','Postcode_Fixed','PostCode1','Outercode']][(df['PostCode1'].str.len()>4) & (df['RegAddress.Country'] == 'UNITED KINGDOM')]

,RegAddress.PostCode,Postcode_FX,Postcode_Fixed,PostCode1,Outercode
3199,SW1X9SH,SW1X9SH,"[(SW1X9SH, SW1X, SW, 1X, 9SH, 9, SH)]",SW1X9SH,SK6 3DY
18475,RH162NL,RH162NL,"[(RH162NL, RH16, RH, 16, 2NL, 2, NL)]",RH162NL,SK6 3DY
18918,BH170GB,BH170GB,"[(BH170GB, BH17, BH, 17, 0GB, 0, GB)]",BH170GB,SK6 3DY
31366,BL13HT,BL13HT,"[(BL13HT, BL1, BL, 1, 3HT, 3, HT)]",BL13HT,SK6 3DY
34253,OX41XX,OX41XX,"[(OX41XX, OX4, OX, 4, 1XX, 1, XX)]",OX41XX,SK6 3DY
82446,WA51DA,WA51DA,"[(WA51DA, WA5, WA, 5, 1DA, 1, DA)]",WA51DA,SK6 3DY
123676,EC2M3TY,EC2M3TY,"[(EC2M3TY, EC2M, EC, 2M, 3TY, 3, TY)]",EC2M3TY,SK6 3DY
124756,NE258EP,NE258EP,"[(NE258EP, NE25, NE, 25, 8EP, 8, EP)]",NE258EP,SK6 3DY
139522,GL69HH,GL69HH,"[(GL69HH, GL6, GL, 6, 9HH, 9, HH)]",GL69HH,SK6 3DY
148515,PL158TL,PL158TL,"[(PL158TL, PL15, PL, 15, 8TL, 8, TL)]",PL158TL,SK6 3DY


In [57]:
df.loc[df['PostCode1'] == 'XXXXXXXXX']

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,CompanyCategory,CompanyStatus,CountryOfOrigin,DissolutionDate,IncorporationDate,Accounts.AccountRefDay,Accounts.AccountRefMonth,Accounts.NextDueDate,Accounts.LastMadeUpDate,Accounts.AccountCategory,Returns.NextDueDate,Returns.LastMadeUpDate,Mortgages.NumMortCharges,Mortgages.NumMortOutstanding,Mortgages.NumMortPartSatisfied,Mortgages.NumMortSatisfied,SICCode.SicText_1,SICCode.SicText_2,SICCode.SicText_3,SICCode.SicText_4,LimitedPartnerships.NumGenPartners,LimitedPartnerships.NumLimPartners,URI,PreviousName_1.CONDATE,PreviousName_1.CompanyName,PreviousName_2.CONDATE,PreviousName_2.CompanyName,PreviousName_3.CONDATE,PreviousName_3.CompanyName,PreviousName_4.CONDATE,PreviousName_4.CompanyName,PreviousName_5.CONDATE,PreviousName_5.CompanyName,PreviousName_6.CONDATE,PreviousName_6.CompanyName,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate,Reg_Country,PostCode1,Postcode_FX,Postcode_Fixed
1021,00722745 LIMITED,00722745,NaN,NaN,XXXXXXXXXXXXXX,XXXXXXXXXXXXXXXXX,XXXXXXXXXXXXXXXXX,NaN,NaN,XXXXXXXXX,Private Limited Company,Active,United Kingdom,NaN,01/05/1962,31.0,5.0,01/03/1964,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00722745,09/01/2018,PAMETRADA LIMITED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXXXXXXXX,XXXXXXXXX,[]
1304847,E. GORDON HILL & SON LIMITED,00538498,NaN,NaN,XXXXXXXXXX XXXXXXXXXXXXXXX,XXXXXXXXXXXXXXX,XXXXXXXXXXXXXXX,XXXXXXXXXXXXXX,NaN,XXXXXXXXX,Private Limited Company,Liquidation,United Kingdom,NaN,01/01/1900,31.0,1.0,01/11/1901,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00538498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15/01/2017,NaN,NaN,XXXXXXXXX,XXXXXXXXX,[]


In [58]:
df.loc[df['PostCode1'] == 'XXXXXXXX']

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,CompanyCategory,CompanyStatus,CountryOfOrigin,DissolutionDate,IncorporationDate,Accounts.AccountRefDay,Accounts.AccountRefMonth,Accounts.NextDueDate,Accounts.LastMadeUpDate,Accounts.AccountCategory,Returns.NextDueDate,Returns.LastMadeUpDate,Mortgages.NumMortCharges,Mortgages.NumMortOutstanding,Mortgages.NumMortPartSatisfied,Mortgages.NumMortSatisfied,SICCode.SicText_1,SICCode.SicText_2,SICCode.SicText_3,SICCode.SicText_4,LimitedPartnerships.NumGenPartners,LimitedPartnerships.NumLimPartners,URI,PreviousName_1.CONDATE,PreviousName_1.CompanyName,PreviousName_2.CONDATE,PreviousName_2.CompanyName,PreviousName_3.CONDATE,PreviousName_3.CompanyName,PreviousName_4.CONDATE,PreviousName_4.CompanyName,PreviousName_5.CONDATE,PreviousName_5.CompanyName,PreviousName_6.CONDATE,PreviousName_6.CompanyName,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate,Reg_Country,PostCode1,Postcode_FX,Postcode_Fixed
962,00593475 LIMITED,00593475,NaN,NaN,XXXXXXXXXX,XXXXXXXXXX,XXXXXXXXXX,XXXXXXXXXX,NaN,XXXXXXXX,Private Limited Company,Active,United Kingdom,NaN,12/11/2013,1.0,1.0,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00593475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXXXXXXX,XXXXXXXX,[]
3578907,ROWAN FABRICATIONS CO LIMITED,01322627,XXXXXXX,NaN,XXXXXXXXXX XXXXXXXXX,XXXXXXXXX,XXXXXXXXXX,XXXXXXXXX,NaN,XXXXXXXX,"PRIV LTD SECT. 30 (Private limited company, section 30 of the Companies Act)",Liquidation,United Kingdom,NaN,22/07/1977,NaN,NaN,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/01322627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXXXXXXX,XXXXXXXX,[]
3581259,ROY ROOFING LIMITED,00396181,NaN,NaN,XXXXXXXXXX,XXXXXXXXXX,XXXXXXXXXX,XXXXXXXXXX,NaN,XXXXXXXX,Private Limited Company,Liquidation,United Kingdom,NaN,13/06/1945,NaN,NaN,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00396181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXXXXXXX,XXXXXXXX,[]
4042766,T.GRAY & CO.LIMITED,00975066,NaN,NaN,XXXXXXXXXX,XXXXXXXXXX,XXXXXXXXXX,XXXXXXXXXX,NaN,XXXXXXXX,Private Limited Company,Liquidation,United Kingdom,NaN,19/03/1970,NaN,NaN,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00975066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXXXXXXX,XXXXXXXX,[]
4340013,TURNER WAREHOUSING LIMITED,00070792,NaN,NaN,XXXXXXX,XXXXXXX,XXXXXXX,XXXXXXX,NaN,XXXXXXXX,Private Limited Company,Active,United Kingdom,NaN,01/01/1900,1.0,1.0,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00070792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15/01/2017,NaN,NaN,XXXXXXXX,XXXXXXXX,[]


In [60]:
df.loc[df['PostCode1'] == 'XXXX']

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,CompanyCategory,CompanyStatus,CountryOfOrigin,DissolutionDate,IncorporationDate,Accounts.AccountRefDay,Accounts.AccountRefMonth,Accounts.NextDueDate,Accounts.LastMadeUpDate,Accounts.AccountCategory,Returns.NextDueDate,Returns.LastMadeUpDate,Mortgages.NumMortCharges,Mortgages.NumMortOutstanding,Mortgages.NumMortPartSatisfied,Mortgages.NumMortSatisfied,SICCode.SicText_1,SICCode.SicText_2,SICCode.SicText_3,SICCode.SicText_4,LimitedPartnerships.NumGenPartners,LimitedPartnerships.NumLimPartners,URI,PreviousName_1.CONDATE,PreviousName_1.CompanyName,PreviousName_2.CONDATE,PreviousName_2.CompanyName,PreviousName_3.CONDATE,PreviousName_3.CompanyName,PreviousName_4.CONDATE,PreviousName_4.CompanyName,PreviousName_5.CONDATE,PreviousName_5.CompanyName,PreviousName_6.CONDATE,PreviousName_6.CompanyName,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate,Reg_Country,PostCode1,Postcode_FX,Postcode_Fixed
1044,00838117 LIMITED,00838117,NaN,NaN,XXXXXXX,XXXXXXX,XXXXXXX,XXXXXXX,NaN,XXXX XXX,Private Limited Company,Active,United Kingdom,NaN,18/02/1965,28.0,2.0,18/12/1966,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00838117,18/02/2014,CHARLES MCCAIN LIMITED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/03/2017,NaN,NaN,XXXX,XXXX XXX,[]
1156,01047395 LIMITED,01047395,NaN,NaN,XXXXXXXXXXXXX,"XXXXXXXXXXXXX, XXXXXXXXXXXX",XXXXXXXXXXXXXXXXXX,"XXXXXXXXXXXXXXXX, XXXXXXXX",NaN,XXXX XXX,Private Limited Company,Active,United Kingdom,NaN,24/03/1972,24.0,3.0,24/01/1974,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/01047395,10/06/2020,TIVERTON CONSTRUCTION LIMITED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXXX,XXXX XXX,[]
736787,C. H. ELKES & SONS LIMITED,00226156,NaN,NaN,XXXXXXX,XXXXXXX,XXXXXXX,NaN,NaN,XXXX XXX,Private Limited Company,Active,United Kingdom,NaN,01/01/1900,31.0,1.0,01/11/1901,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00226156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXXX,XXXX XXX,[]
2079240,INSTANT HEAT (BIRMINGHAM) LIMITED,00945592,NaN,NaN,XX,XXXXXXX,XXXXXXX,XXXXXXX,UNITED KINGDOM,XXXX,Private Limited Company,Active,United Kingdom,NaN,01/01/1969,31.0,1.0,01/11/1970,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00945592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15/01/2017,NaN,UNITED KINGDOM,XXXX,XXXX,[]
3055881,OLDHAM ROPE AND TWINE COMPANY (THE) LIMITED,00172971,NaN,NaN,XXXX,XXXX,XXXX,XXXX,NaN,XXXX,Private Limited Company,Liquidation,United Kingdom,NaN,01/01/1919,31.0,1.0,01/11/1920,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00172971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXXX,XXXX,[]
3410539,R. CLIFFORD LIMITED,00565506,NaN,NaN,XXXX,NaN,XXXX,NaN,NaN,XXXX,Private Limited Company,Active,United Kingdom,NaN,01/05/1956,NaN,NaN,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00565506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXXX,XXXX,[]


In [61]:
df.loc[df['PostCode1'] == 'XXX']

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,CompanyCategory,CompanyStatus,CountryOfOrigin,DissolutionDate,IncorporationDate,Accounts.AccountRefDay,Accounts.AccountRefMonth,Accounts.NextDueDate,Accounts.LastMadeUpDate,Accounts.AccountCategory,Returns.NextDueDate,Returns.LastMadeUpDate,Mortgages.NumMortCharges,Mortgages.NumMortOutstanding,Mortgages.NumMortPartSatisfied,Mortgages.NumMortSatisfied,SICCode.SicText_1,SICCode.SicText_2,SICCode.SicText_3,SICCode.SicText_4,LimitedPartnerships.NumGenPartners,LimitedPartnerships.NumLimPartners,URI,PreviousName_1.CONDATE,PreviousName_1.CompanyName,PreviousName_2.CONDATE,PreviousName_2.CompanyName,PreviousName_3.CONDATE,PreviousName_3.CompanyName,PreviousName_4.CONDATE,PreviousName_4.CompanyName,PreviousName_5.CONDATE,PreviousName_5.CompanyName,PreviousName_6.CONDATE,PreviousName_6.CompanyName,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate,Reg_Country,PostCode1,Postcode_FX,Postcode_Fixed
932,00527065 LTD,00527065,NaN,NaN,XXX XXX,XXX,XXX,XXX,NaN,XXX,Private Limited Company,Active,United Kingdom,NaN,01/01/1901,31.0,1.0,01/11/1902,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00527065,31/01/2014,MICHAEL MCKENNA LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15/01/2017,NaN,NaN,XXX,XXX,[]
960482,COLIN MACANDREW AND PARTNERS LIMITED (77/1420),SC018074,NaN,NaN,XXXXXXXX,NaN,XXXXX,NaN,NaN,XXX XXX,Private Limited Company,Active,United Kingdom,NaN,29/07/1974,1.0,1.0,01/11/1975,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/SC018074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXX,XXX XXX,[]
1480108,F.DEELEY LIMITED,00361481,NaN,NaN,XXXX,NaN,XXXX,NaN,NaN,XXX XXX,Private Limited Company,Active,United Kingdom,NaN,01/01/1939,1.0,1.0,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00361481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXX,XXX XXX,[]
1846584,HANCOCK BROS.(CAMBRIDGE)LIMITED,00720967,NaN,NaN,XXXXXXXX,NaN,XXXXX,NaN,NaN,XXX XXX,Private Limited Company,Active,United Kingdom,NaN,09/04/1964,NaN,NaN,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00720967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXX,XXX XXX,[]
2166062,JAMES KERSLAKE LIMITED,00333041,NaN,NaN,XXXXXX,XXXXXX,XXXXXX,NaN,NaN,XXX XXX,Private Limited Company,Liquidation,United Kingdom,NaN,01/01/1900,NaN,NaN,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00333041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXX,XXX XXX,[]
4041830,T.& A.NAYLOR LIMITED,00112983,NaN,NaN,XXX XXX,XXX,XXX,XXX,NaN,XXX,Private Limited Company,Active,United Kingdom,NaN,30/11/1910,30.0,11.0,30/09/1912,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00112983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14/12/2016,NaN,NaN,XXX,XXX,[]
4584064,WM. DRABBLE & SONS LIMITED,00653248,NaN,NaN,XXXXXX,XXXXXX,XXXXXX,NaN,NaN,XXX XXX,Private Limited Company,Active,United Kingdom,NaN,21/03/1960,NaN,NaN,NaN,NaN,NO ACCOUNTS FILED,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/00653248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXX,XXX XXX,[]


In [ ]:
df[some_column] == some_value 
df[some_column].isin(some_list_of_values) # This check whether the value of the column is one of the values in the list
df[some_column].str.contains() # You can use it the same as str.contains()
df[some_column].str.isdigit() # Same usage as str.isdigit(), check whether string is all digits, need to make sure column type is string in advance
df[some_column].str.len() == 4 # Filter string with length of 4